In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/linregonlinelearn/other/default/1/saved_model_lr_ol_gpu.pkl
/kaggle/input/jane-street-real-time-market-data-forecasting/responders.csv
/kaggle/input/jane-street-real-time-market-data-forecasting/sample_submission.csv
/kaggle/input/jane-street-real-time-market-data-forecasting/features.csv
/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=4/part-0.parquet
/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=5/part-0.parquet
/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=6/part-0.parquet
/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=3/part-0.parquet
/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=1/part-0.parquet
/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=8/part-0.parquet
/kaggle/input/jane-street-real-time-market-data-forecasting/train.parque

In [2]:
import os

import pandas as pd
import polars as pl

import kaggle_evaluation.jane_street_inference_server

In [3]:
import pickle
import polars as pl
import pandas as pd
from cuml.linear_model import LinearRegression as cuLinearRegression

# Global variables to store lags and the loaded model
lags_ = None
loaded_model = None

def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction and update the model using online learning."""
    global lags_, loaded_model

    # Initialize or update lags
    if lags is not None:
        lags_ = lags

    # Load the saved model if not already loaded
    if loaded_model is None:
        with open("/kaggle/input/linregonlinelearn/other/default/1/saved_model_lr_ol_gpu.pkl", "rb") as f:
            loaded_model = pickle.load(f)

    # Combine test features with lags
    combined_data = test.join(lags_, on=["date_id", "time_id", "symbol_id"], how="left")
    combined_data = combined_data.with_columns(
        pl.all().fill_null(strategy="forward")
    )

    # Define the features to use for prediction
    feature_columns = [
        'responder_3_lag_1', 'responder_8_lag_1', 'responder_7_lag_1', 'responder_4_lag_1', 'responder_5_lag_1',
        'responder_0_lag_1', 'responder_2_lag_1', 'responder_1_lag_1', 
        'feature_06', 'feature_60', 'feature_49', 'feature_04', 'feature_07', 
        'feature_58', 'feature_59', 'feature_47', 'feature_51', 'feature_36', 
        'feature_52', 'feature_68', 'feature_13', 'feature_02', 'feature_05', 
        'feature_41', 'feature_01', 'time_id', 'feature_54', 'feature_40', 
        'feature_03', 'feature_55', 'feature_08', 'feature_19', 'feature_48', 
        'feature_00', 'feature_71', 'feature_66', 'feature_45'
    ]

    # Prepare the input features
    X_pred = combined_data.select(feature_columns).to_pandas()
    X_pred.fillna(0, inplace=True)

    # Predict using the loaded model
    predictions = loaded_model.predict(X_pred)

    # Update the model using the current batch for online learning
    if 'responder_6' in combined_data.columns:
        X_train = X_pred  # The same features are used for training
        y_train = combined_data['responder_6'].to_pandas()

        # Incremental training (online learning)
        loaded_model.fit(X_train, y_train)

        # Save the updated model
        with open("/kaggle/working/saved_model_lr_ol_gpu.pkl", "wb") as f:
            pickle.dump(loaded_model, f)

    # Add predictions to the DataFrame
    combined_data = combined_data.with_columns(pl.Series(name="responder_6", values=predictions))

    # Return only the row_id and predictions as required
    predictions = combined_data.select(
        'row_id', 'responder_6'
    )

    # Ensure the return format matches expectations
    if isinstance(predictions, pl.DataFrame):
        assert predictions.columns == ['row_id', 'responder_6']
    elif isinstance(predictions, pd.DataFrame):
        assert (predictions.columns == ['row_id', 'responder_6']).all()
    else:
        raise TypeError('The predict function must return a DataFrame')

    # Ensure the number of predictions matches the input test data
    assert len(predictions) == len(test)

    return predictions


In [4]:
inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet',
            '/kaggle/input/jane-street-real-time-market-data-forecasting/lags.parquet',
        )
    )

In [5]:
import os
if os.path.isfile('submission.parquet'):
    pl_sub = pl.read_parquet('submission.parquet')
    display(pl_sub)

row_id,responder_6
i64,f32
0,0.003904
1,0.023779
2,0.002901
3,0.004011
4,0.011738
…,…
34,0.001894
35,0.010364
36,-0.004901
